In [70]:
import yfinance as yf
import pandas as pd
from IPython.display import display, clear_output

In [66]:
def get_tickers(file=r"Stock names.txt"):
    with open(file) as f:
        tickers = f.read().split("\n")
    return [ticker for ticker in tickers if "." not in ticker]
    return tickers

In [62]:
def get_basic_info(stock):
    # print(ticker, end="' ")
    # stock = yf.Ticker(ticker)
    data = stock.info
    div_yield = data["dividendYield"]  # * 100
    div_ex = pd.to_datetime(data["exDividendDate"], unit='s')  # Ex date
    payout = data["payoutRatio"] # * 100
    trail_div_yield = data["trailingAnnualDividendYield"]  # * 100
    avg_yield_5y = data["fiveYearAvgDividendYield"]
    
    return div_yield, trail_div_yield, div_ex, payout, avg_yield_5y

In [63]:
def get_div_info(stock):
    tmp = stock.dividends
    div_df = pd.DataFrame(index=tmp.index)
    div_df["Dividends"] = tmp
    div_df.index = div_df.index.year
    div_df = div_df.groupby(div_df.index).mean()
    div_df["Growth year"] = ((div_df["Dividends"]/div_df["Dividends"].shift(1)) - 1) * 100
    div_df["DGR"] = div_df["Growth year"].expanding().mean()
    div_df["DGR5y"] = div_df["Growth year"].rolling(5).mean()
    div_df["DGR3y"] = div_df["Growth year"].rolling(3).mean()
    div_df["DGR10y"] = div_df["Growth year"].rolling(10).mean()
    div_df["Years grow"] = div_df.index - div_df.first_valid_index()
    
    growth_rate = float(div_df.tail(1)["DGR"])
    growth_year = int(div_df.tail(1)["Years grow"])  # Years of growth
    growth_rate_3 = float(div_df.tail(1)["DGR3y"])
    growth_rate_5 = float(div_df.tail(1)["DGR5y"])
    growth_rate_10 = float(div_df.tail(1)["DGR10y"])
    
    return growth_rate, growth_year, growth_rate_3, growth_rate_5, growth_rate_10

In [85]:
def main():
    tickers = get_tickers()
    # tickers = ["CCMP", "O"]
    tickers = yf.Tickers(tickers)
    df = pd.DataFrame(columns=["div yield", "Trailing annual div yield", "exDiv", "payout ratio", "5 year yield average", "DGR", "Years of grow", "DGR3y", 
                          "DGR5y", "DGR10y"])
    
    for ticker in tickers.tickers:
        # print(ticker, end=", ")
        stock = yf.Ticker(ticker)
        fail = False
        while True:
            try:
                basic_info = get_basic_info(stock)
                div_info = get_div_info(stock)
            except TypeError:
                fail = True
                break
            except ConnectionError:
                print("Connection error")
                # time.sleep(15)
                continue
            break
        if fail:
            continue
        df.loc[ticker] = basic_info[0], basic_info[1], basic_info[2], basic_info[3], basic_info[4], div_info[0], div_info[1], div_info[2], div_info[3], div_info[4]
        clear_output(wait=True)
        display(df)

    df["payout ratio"] *= 100
    df["div yield"] *= 100
    df["Trailing annual div yield"] *= 100
    
    return df

In [ ]:
df = main()

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,DGR,Years of grow,DGR3y,DGR5y,DGR10y
A,0.0058,0.005377,2022-12-30,0.2010,0.72,1.918650,16,8.238150,9.361635,11.624368
AAPL,0.0065,0.006252,2022-11-04,0.1473,0.98,468.068384,35,6.191018,8.201724,9.197138
ABBV,0.0402,0.038263,2023-01-12,0.7373,4.36,14.391911,10,7.865190,10.619200,14.391911
ABC,0.0118,0.011158,2022-11-10,0.2289,1.62,60.429180,21,5.243721,4.806809,12.579866
ABM,0.0192,0.016982,2023-01-04,0.2287,1.92,6.402326,39,6.051598,4.758662,3.859558
...,...,...,...,...,...,...,...,...,...,...
GBCI,0.0247,0.026293,2022-10-07,0.6708,2.49,8.035953,36,1.252802,3.611530,7.458605
GBX,0.0391,0.037396,2022-11-07,0.6243,2.91,15.644545,28,2.666667,4.280891,28.811938
GCBC,0.0086,0.010345,2022-11-14,0.1510,1.44,8.638287,21,6.644081,5.063837,3.603233
GD,0.0205,0.020473,2022-10-06,0.4153,2.26,36.001353,43,6.099908,7.777562,9.049569
